In [1]:
import pandas as pd
from faker import Faker
import random
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

c:\Users\DESKTOP-1\Desktop\GenAI-Projects\genai-venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [71]:
# Initialize Faker
fake = Faker()

# List of 20 items with their prices
items = {
    'milk': 5,
    'bread': 3,
    'eggs': 2,
    'cheese': 4,
    'chocolate': 6,
    'coffee': 8,
    'tea': 4,
    'apple': 1,
    'banana': 2,
    'orange': 2,
    'tomato': 1,
    'potato': 1,
    'carrot': 1,
    'chicken': 10,
    'beef': 12,
    'fish': 9,
    'rice': 3,
    'pasta': 4,
    'pizza': 7,
    'burger': 6
}

# Define the number of samples
number_of_samples = 40000  # Change this value as needed

# Generate fake data
data = {
    'gender': [random.choice(['female', 'male']) for _ in range(number_of_samples)],
    'transaction_date': [fake.date_time_this_year() for _ in range(number_of_samples)],  # Using date_time_this_year to include time
    'amount': [fake.random_number(digits=4) for _ in range(number_of_samples)],
    'item_name': [random.choice(list(items.keys())) for _ in range(number_of_samples)],
    'amount_purchased': [random.randint(1, 10) for _ in range(number_of_samples)],
    'acc_balance': [fake.random_number(digits=5) for _ in range(number_of_samples)]
}

# Calculate item price, profit, and extract hour
data['item_price'] = [items[item] for item in data['item_name']]
data['profit'] = [price * amount for price, amount in zip(data['item_price'], data['amount_purchased'])]
data['transaction_hour'] = [date.hour for date in data['transaction_date']]

# Create DataFrame
df = pd.DataFrame(data)

df

,gender,transaction_date,amount,item_name,amount_purchased,acc_balance,item_price,profit,transaction_hour
0,male,2024-02-27 14:18:24,5467,cheese,5,38936,4,20,14
1,male,2024-04-19 06:32:04,8723,rice,10,26871,3,30,6
2,female,2024-02-14 15:08:51,8575,apple,7,70229,1,7,15
3,female,2024-03-02 17:49:04,1279,tea,7,67129,4,28,17
4,female,2024-04-10 21:34:34,4719,carrot,9,70394,1,9,21
...,...,...,...,...,...,...,...,...,...
39995,male,2024-01-20 17:42:00,7472,eggs,2,25838,2,4,17
39996,female,2024-04-24 15:45:12,150,pizza,2,89378,7,14,15
39997,female,2024-02-22 00:00:22,7324,pasta,7,61685,4,28,0
39998,male,2024-01-03 03:44:09,5894,coffee,9,16588,8,72,3


## Using LLM to Generate Insights

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                              trust_remote_code=True,
                                              revision="main",)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

c:\Users\DESKTOP-1\Desktop\GenAI-Projects\genai-venv\lib\site-packages\transformers\modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [65]:
def return_prompt(data,meta_data=''):

    return f"""<<SYS>>You are an assistant for a large supermarket<<</SYS>>>
[INST]from the provided report of {meta_data}:
{data}

Provide one action to take to increase revenue based on the above report[/INST]
Answer:\n"""

def generate_insight(data,meta_data=''):
    
    pipe = pipeline(
    "text-generation",
     model=model,
     tokenizer=tokenizer,
     max_new_tokens=2056,
     do_sample=True,
     temperature=0.25,
     top_k=50,
     repetition_penalty=1.5
    )
    generated_text = pipe(return_prompt(data,meta_data))[0]['generated_text']
    result_index = generated_text.find("Answer:") + len("Answer:")
    return generated_text[result_index:].strip()

In [24]:
print(return_prompt('test'))

<<SYS>>You are an assistant for a large supermarket<<</SYS>>>
[INST]from the provided report:
test

Provide one action to take to increase revenue based on the above report[/INST]
Answer:



In [44]:
# Aggregate total amount spent on each item by gender
total_amount_by_item_gender = df.groupby(['gender', 'item_name'])['profit'].sum().reset_index().sort_values(by='profit',ascending=False)
result = total_amount_by_item_gender.to_string()
print(result)

    gender  item_name  profit
22    male       beef   65028
2   female       beef   60972
7   female    chicken   55950
27    male    chicken   55190
11  female       fish   48294
31    male       fish   46737
29    male     coffee   43568
9   female     coffee   42040
35    male      pizza   39725
15  female      pizza   39396
8   female  chocolate   34764
24    male     burger   34374
4   female     burger   32778
28    male  chocolate   32340
32    male       milk   29060
12  female       milk   27685
18  female        tea   23660
38    male        tea   22844
6   female     cheese   22352
34    male      pasta   22268
26    male     cheese   21884
14  female      pasta   21628
3   female      bread   16551
17  female       rice   16533
23    male      bread   16443
37    male       rice   15984
21    male     banana   11650
1   female     banana   11302
10  female       eggs   11032
33    male     orange   10900
30    male       eggs   10732
13  female     orange   10614
19  female

In [45]:
generate_insight(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, we can observe that "beef" and "chicken" items have higher profits compared to other food categories. Therefore, increasing sales of these two specific meat products could potentially lead to increased revenues in thissuper market. One possibleactioncould be running targeted promotions or advertisements focusingonthesemeatitemstoencouragecustomers\'purchasesandboostrevenuethroughincreasedsalesvolumeofhigher-profitproducts.'

### Generated Insight:

Based on the given data, we can observe that "beef" and "chicken" items have higher profits compared to other food categories. Therefore, increasing sales of these two specific meat products could potentially lead to increased revenues. One possible way to do this is by promoting special deals or discounts specifically for those items during certain periods (e.g., weekends) or creating targeted marketing campaigns towards customers who frequently purchase them.

In [46]:
# Find the gender that purchased the most of each item
max_purchase_by_item_gender = df.groupby(['gender', 'item_name'])['amount_purchased'].sum().reset_index()
max_purchase_by_item_gender = max_purchase_by_item_gender.sort_values(by='amount_purchased', ascending=False).groupby('item_name').head(1)
result = max_purchase_by_item_gender.to_string()
print(max_purchase_by_item_gender)

    gender  item_name  amount_purchased
18  female        tea              5915
21    male     banana              5825
32    male       milk              5812
8   female  chocolate              5794
24    male     burger              5729
35    male      pizza              5675
19  female     tomato              5642
7   female    chicken              5595
0   female      apple              5588
6   female     cheese              5588
34    male      pasta              5567
3   female      bread              5517
10  female       eggs              5516
17  female       rice              5511
36    male     potato              5488
33    male     orange              5450
29    male     coffee              5446
5   female     carrot              5445
22    male       beef              5419
11  female       fish              5366


In [48]:
generate_insight(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, we can identify that customers purchased various items with different quantities. One potential strategy to Increase Revenue could be focusing on increasing sales of high-value or frequently demanded products such as "milk," "bananas," and "chicken." This might involve promoting these items through targeted marketing campaigns, offering discounts or bundles, improving their placement in stores, etc. However, it\'s important to note this is just one possible approach; further analysis considering factors like customer demographics, seasonality, inventory levels would provide more accurate insights before making any decisions.'

### Generated Insight:

Based on the given data, we can identify that customers purchased various items with different quantities. One potential strategy to Increase Revenue could be focusing on increasing sales of high-value or frequently demanded products such as "milk," "bananas," and "chicken." This might involve promoting these items through targeted marketing campaigns, offering discounts or bundles, improving their placement in stores, etc. However, it\'s important to note this is just one possible approach; further analysis considering factors like customer demographics, seasonality, inventory levels would provide more accurate insights before making any decisions.

In [54]:
# Calculate the average purchase amount by gender
average_purchase_by_gender = df.groupby('gender')['amount_purchased'].mean().reset_index()
result = average_purchase_by_gender.to_string()
print(average_purchase_by_gender)


   gender  amount_purchased
0  female          5.502223
1    male          5.511434


In [55]:
generate_insight(result,'average purchase amount by gender')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, it appears that there is only a slight difference in the average purchasing amounts between males and females at this specific store or chain. To potentially increase overall revenue without favoring either group specifically, consider implementing strategies such as upselling, cross- selling, promotions, discounts, loyalty programs, or improving customer experience across all genders equally. These tactics can encourage larger purchases from both groups while maintaining fairness towards customers regardless of their gender.'

### Generated Insight:

Based on the given data, it appears that there is only a slight difference in the average purchasing amounts between males and females at this specific store or chain. To potentially increase overall revenue without favoring either group specifically, consider implementing strategies such as upselling, cross- selling, promotions, discounts, loyalty programs, or improving customer experience across all genders equally. These tactics can encourage larger purchases from both groups while maintaining fairness towards customers regardless of their gender.

In [61]:
# Calculate the total amount spent by each gender overall
total_amount_by_gender = df.groupby('gender')['profit'].sum().reset_index().sort_values(by='profit',ascending=False)
result = total_amount_by_gender.to_string()
print(total_amount_by_gender)


   gender  profit
1    male  500491
0  female  497547


In [67]:
print(generate_insight(result))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Based on the given data, there is only a small difference in profits between males and females. One potential strategy could be targeting marketing efforts towards increasing sales among Female customers by offering promotions or discounts specifically tailored to their preferences. This approach may help attract more women shoppers and potentially lead to increased overall revenues without significantly impacting men's spending habits. However, it would also require further research into understanding what drives purchasing decisions differently between genders within your customer base before implementing any targeted campaigns effectively.


### Generated Insight

Based on the given data, there is only a small difference in profits between males and females. One potential strategy could be targeting marketing efforts towards increasing sales among Female customers by offering promotions or discounts specifically tailored to their preferences. This approach may help attract more women shoppers and potentially lead to increased overall revenues without significantly impacting men's spending habits. However, it would also require further research into understanding what drives purchasing decisions differently between genders within your customer base before implementing any targeted campaigns effectively

## Getting Insights with timelines

In [76]:
# Convert 'transaction_date' to datetime format if it's not already
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# Extract the hour from the transaction_date
df['transaction_hour'] = df['transaction_date'].dt.hour

# Count the number of transactions per hour
transactions_per_hour = df.groupby('transaction_hour')['gender'].count().reset_index()
max_buyers_hour = transactions_per_hour[transactions_per_hour['gender'] == transactions_per_hour['gender'].max()]
result = max_buyers_hour.to_string()
print(max_buyers_hour)


   transaction_hour  gender
9                 9    1789


In [77]:
generate_insight(result,'most hour of the day we got sales')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, it appears that there is high demand and potentially significant sales during the ninth hour. To capitalize on this trend and possibly boost revenues further, consider implementing promotions or discounts specifically during that time frame. This could attract more customers and encourage larger purchases from existing ones in addition to those already making transactions at that hour. Additionally, ensuring adequate staffing and inventory availability can help optimally serve these shoppers and maintain positive customer experiences.'

### Generated Insight

Based on the given data, it appears that there is high demand and potentially significant sales during the ninth hour. To capitalize on this trend and possibly boost revenues further, consider implementing promotions or discounts specifically during that time frame. This could attract more customers and encourage larger purchases from existing ones in addition to those already making transactions at that hour. Additionally, ensuring adequate staffing and inventory availability can help optimally serve these shoppers and maintain positive customer experiences.

In [79]:
# Calculate the total amount spent per hour
total_amount_per_hour = df.groupby('transaction_hour')['profit'].sum().reset_index()
max_total_amount_hour = total_amount_per_hour[total_amount_per_hour['profit'] == total_amount_per_hour['profit'].max()]
print(max_total_amount_hour)


   transaction_hour  profit
6                 6   45845


In [80]:
generate_insight(max_total_amount_hour.to_string())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, it appears that there is significant sales volume and profits during hour 6. To capitalize on this trend, consider increasing inventory levels or promoting popular items specifically during that time frame in order to maximize revenues. This could be achieved through various methods such as staffing more cashiers, offering promotions, or adjusting pricing strategies accordingly.'

### Generated Insight

Based on the given data, it appears that there is significant sales volume and profits during hour 6. To capitalize on this trend, consider increasing inventory levels or promoting popular items specifically during that time frame in order to maximize revenues. This could be achieved through various methods such as staffing more cashiers, offering promotions, or adjusting pricing strategies accordingly.

In [81]:
# Calculate the number of transactions per hour by gender
transactions_per_hour_gender = df.groupby(['gender', 'transaction_hour'])['gender'].count().unstack(fill_value=0)
print(transactions_per_hour_gender)


transaction_hour   0    1    2    3    4    5    6    7    8    9   ...   14  \
gender                                                              ...        
female            820  817  860  839  816  861  870  833  775  897  ...  840   
male              800  838  822  868  869  836  874  869  851  892  ...  823   

transaction_hour   15   16   17   18   19   20   21   22   23  
gender                                                         
female            796  872  796  823  803  830  839  824  827  
male              886  788  797  847  819  820  793  845  779  

[2 rows x 24 columns]


In [82]:
generate_insight(transactions_per_hour_gender.to_string())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, it appears that there is a higher number of female customers shopping during hours 13-15 (lunchtime) and males tend to shop more in hour 18. To Increase overall revenues, consider targeting marketing efforts towards increasing male customer traffic between hours 16-19 when they seem less likely to be present according to this dataset. This could include special promotions or discounts specifically targeted at men during these times. Additionally, considering extending business hours slightly past midnight if sales trends indicate significant potential demand from late night shoppers across both genders.'

### Generated Insight

Based on the given data, it appears that there is a higher number of female customers shopping during hours 13-15 (lunchtime) and males tend to shop more in hour 18. To Increase overall revenues, consider targeting marketing efforts towards increasing male customer traffic between hours 16-19 when they seem less likely to be present according to this dataset. This could include special promotions or discounts specifically targeted at men during these times. Additionally, considering extending business hours slightly past midnight if sales trends indicate significant potential demand from late night shoppers across both genders.

#### Looking for questions in days instead of hours

In [109]:
# Define a mapping dictionary for day of the week
day_mapping = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

In [112]:
# Extract the day of the week from the transaction_date and map it to text
df['transaction_day_of_week'] = df['transaction_date'].dt.dayofweek.map(day_mapping)

# Calculate total profit per day of the week
total_profit_per_day = df.groupby('transaction_day_of_week')['profit'].sum().reset_index()
max_profit_day = total_profit_per_day[total_profit_per_day['profit'] == total_profit_per_day['profit'].max()]
print(max_profit_day)

  transaction_day_of_week  profit
2                Saturday  146148


In [85]:
generate_insight(max_profit_day.to_string())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, it appears that there is significant sales volume and profits being generated on Thursdays. One potential strategy could be focusing marketing efforts or promotions specifically towards this day of the week in order to attract even more customers and potentially boost overall revenues further. This might include targeted advertising campaigns, special discounts or deals exclusive to Thursday shoppers, or other initiatives aimed at increasing foot traffic during this time period. However, additional analysis may also be necessary to ensure such strategies would indeed lead to increased net income rather than just higher expenses due to added promotion costs.'

### Generated Insight

Based on the given data, it appears that there is significant sales volume and profits being generated on Thursdays. One potential strategy could be focusing marketing efforts or promotions specifically towards this day of the week in order to attract even more customers and potentially boost overall revenues further. This might include targeted advertising campaigns, special discounts or deals exclusive to Thursday shoppers, or other initiatives aimed at increasing foot traffic during this time period. However, additional analysis may also be necessary to ensure such strategies would indeed lead to increased net income rather than just higher expenses due to added promotion costs.

In [115]:
# Calculate total profit per day of the week
total_profit_per_day = df.groupby('transaction_day_of_week')['profit'].sum().reset_index().sort_values(by='profit',ascending=False)
# Set the day of the week as the index
total_profit_per_day.set_index('transaction_day_of_week', inplace=True)
print(total_profit_per_day)

                         profit
transaction_day_of_week        
Saturday                 146148
Monday                   145011
Thursday                 143471
Tuesday                  143176
Wednesday                141909
Sunday                   140253
Friday                   139913


In [116]:
generate_insight(total_profit_per_day.to_string())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, it appears that Saturday has the highest sales volume among all days. To potentially Increase overall revenues, consider implementing promotional strategies or special offers specifically targeting customers during weekdays when sale volumes seem lower compared to Saturdays (i.e., Monday through Friday). This could include discounts, loyalty programs, bundled deals etc.. The goal is to encourage more transactions and larger basket sizes throughout the entire week rather than just relying heavily upon weekend business.'

: 

### Generated Insight

Based on the given data, it appears that Saturday has the highest sales volume among all days. To potentially Increase overall revenues, consider implementing promotional strategies or special offers specifically targeting customers during weekdays when sale volumes seem lower compared to Saturdays (i.e., Monday through Friday). This could include discounts, loyalty programs, bundled deals etc.. The goal is to encourage more transactions and larger basket sizes throughout the entire week rather than just relying heavily upon weekend business.

In [89]:
# Calculate average profit per transaction for each day of the week
average_profit_per_transaction = df.groupby('transaction_day_of_week')['profit'].agg(['mean']).reset_index()
average_profit_per_transaction.columns = ['transaction_day_of_week', 'average_profit']
print(average_profit_per_transaction)


   transaction_day_of_week  average_profit
0                        0       25.298500
1                        1       25.092184
2                        2       24.804929
3                        3       24.813386
4                        4       24.780907
5                        5       25.337725
6                        6       24.845527


In [90]:
generate_insight(average_profit_per_transaction.to_string())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Based on the given data, it appears that there is no significant difference in average profit between different days of the week. However, we do see slightly higher profits on Thursdays (an average profit of $25.33) compared to other weeks' averages ($24.8-25.3). Therefore, focusing marketing efforts or promotions specifically towards Thursday sales could potentially lead to increased revenues without requiring substantial changes to overall operations. This strategy may help attract more customers and boost sales during this day while maintaining consistent weekly performance levels across all seven days."

### Generated Insight

Based on the given data, it appears that there is no significant difference in average profit between different days of the week. However, we do see slightly higher profits on Thursdays (an average profit of $25.33) compared to other weeks' averages ($24.8-25.3). Therefore, focusing marketing efforts or promotions specifically towards Thursday sales could potentially lead to increased revenues without requiring substantial changes to overall operations. This strategy may help attract more customers and boost sales during this day while maintaining consistent weekly performance levels across all seven days.

In [98]:
# Calculate average profit per transaction for each day of the week
average_profit_per_transaction = df.groupby('transaction_day_of_week')['profit'].mean().reset_index()
average_profit_per_transaction.columns = ['transaction_day_of_week', 'profit_mean']
max_average_profit_day = average_profit_per_transaction[average_profit_per_transaction['profit_mean'] == average_profit_per_transaction['profit_mean'].max()]
print(max_average_profit_day)


   transaction_day_of_week  profit_mean
5                        5    25.337725


In [103]:
generate_insight(max_average_profit_day.to_string(),'max mean among days')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, focusing on increasing sales during weekdays (specifically Thursday) could potentially lead to higher revenues as it has shown a relatively high average profit compared to other days in the reported time frame. Strategies such as promoting specific products or offering discounts and special deals exclusively for Thursdays can be considered to attract more customers and boost profits on that day. However, further analysis might be required before implementing any major changes to validate these findings accurately.'

### Generated Insights

Based on the given data, focusing on increasing sales during weekdays (specifically Thursday) could potentially lead to higher revenues as it has shown a relatively high average profit compared to other days in the reported time frame. Strategies such as promoting specific products or offering discounts and special deals exclusively for Thursdays can be considered to attract more customers and boost profits on that day. However, further analysis might be required before implementing any major changes to validate these findings accurately.

In [107]:
# Count the number of transactions per day of the week
transactions_per_day = df.groupby('transaction_day_of_week')['profit'].count().reset_index().sort_values(by='profit',ascending=False)
transactions_per_day.columns = ['transaction_day_of_week', 'count_profit']
print(transactions_per_day)


   transaction_day_of_week  count_profit
3                        3          5782
5                        5          5768
0                        0          5732
2                        2          5721
1                        1          5706
4                        4          5646
6                        6          5645


In [108]:
generate_insight(transactions_per_day.to_string())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Based on the given data, it seems that transactions occurring during certain days of the week (specifically Thursdays and Saturdays with five transactions each) generate relatively high profits. To potentially boost revenues, consider focusing marketing efforts or promotions towards those specific day(s). This could include offering special discounts, running targeted ads, or providing incentives for customers to shop on these profitable days. However, further analysis may be required to confirm this trend holds consistently over time before implementing any major changes in strategy.'